# Setup

## Setup - Imports

In [1]:
# python base imports
import datetime
import json

# import six
import six

print( "packages imported at " + str( datetime.datetime.now() ) )

packages imported at 2022-05-10 01:22:56.361553


## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [2]:
%run ../django_init.py

django initialized at 2022-05-10 01:23:00.154384


In [3]:
# django imports
from django.contrib.auth.models import User

# sourcenet imports
from context_text.shared.context_text_base import ContextTextBase

# context_analysis imports
from context_analysis.network.network_person_info import NetworkPersonInfo

# sourcenet imports
from context_text.models import Article
from context_text.models import Article_Author
from context_text.models import Article_Data
from context_text.models import Article_Subject
from context_text.models import Newspaper
from context_text.models import Person

# article coding
from context_text.article_coding.article_coder import ArticleCoder
#from context_text.article_coding.article_coding import ArticleCoding
from context_text.article_coding.open_calais_v2.open_calais_v2_article_coder import OpenCalaisV2ArticleCoder

# import class that actually processes requests for outputting networks.
from context_text.export.network_output import NetworkOutput

# context_text shared
from context_text.shared.context_text_base import ContextTextBase

print( "django model packages imported at " + str( datetime.datetime.now() ) )

django model packages imported at 2022-05-10 01:23:01.620767


## Setup - Important instances

In [4]:
# get ArticleCoding instance.
#article_coding = ArticleCoding()

# automated coding user
automated_coder = ArticleCoder.get_automated_coding_user()

# newspapers for Grand Rapids Press and Detroit News.
grand_rapids_press = Newspaper.objects.get( newsbank_code = "GRPB" )
detroit_news = Newspaper.objects.get( newsbank_code = "DTNB" )

# Moved to other notebooks

## programmatic network data construction

- make code in [Network Builder](https://research.local/research/context/text/output/network) callable with a JSON dictionary rather than through view.

    - Switched all code into NetworkOutput class (`context_text/export/network_output.py --> NetworkOutput`), got it to work either in view function or stand-alone code with JSON parameters.
    - Example notebook: [analysis-network_data_output_example.ipynb](./analysis-network_data_output_example.ipynb)

## Article overview

- [analysis-article_overview.ipynb](./analysis-article_overview.ipynb)

## Article_Data overview

- [analysis-article_data_overview.ipynb](./analysis-article_data_overview.ipynb)